In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa
from scipy.stats import binom_test
from matplotlib.markers import TICKDOWN
from matplotlib import rcParams
import datetime
from scipy.stats import norm, kstest, sem
from numpy.polynomial import polynomial

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
from lib.file.ROI_reader import ROIFileReader
from lib.analysis.cell_roi import OverlapCounterROI

In [2]:
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/scnn1a-ppr/"
master_csv = data_dir + 'scnn1a-ppr.csv'
###################################
# Process paired-pulse ratio data
#    - IPIs are shuffled, open <slice>_<loc>shuffle.txt for IPI order
#    - extract Amplitude, half_width, latency for pulse 1 and pulse 2 for each ROI
#    - ROIs are in order (in file "<slice>_barrel_rois.dat" for each date subdir):
#          -  medial -> lateral if available
#          - otherwise, in some order that preserves neighbors (1,2 are neighbors and 2,3 are neighbors)
#    - which barrel is home barrel (stimulated) is recorded in master csv scnn1a-ppr.csv for each loc
###################################
enable_photoZ_interact = True
initialize_photoZ = True

snr_cutoff_signal = 0
amp_cutoff = 0

In [3]:
# set up PhotoZ (open it manually)
aPhz = AutoPhotoZ(data_dir=data_dir)
if initialize_photoZ and enable_photoZ_interact:
    aPhz.prepare_photoZ()

In [4]:
# load master csv file for metadata
master_df = pd.read_csv(master_csv)
master_df = pd.DataFrame(master_df)
master_df

,Date,Slice_Loc,Medial Dir,Stim Layer,Caudal/Rostral,Home Barrel
0,12/8/2023,3_2,up,L2/3,Caudal,2
1,12/8/2023,4_1,NaN,L4,Rostral,2
2,12/8/2023,5_1,NaN,L4,Rostral,1
3,12/8/2023,5_2,NaN,L4,Rostral,2
4,12/8/2023,6_1,NaN,L4,NaN,1
5,12/8/2023,6_2,NaN,L4,NaN,2
6,12/13/2023,1_4,NaN,L2/3,Rostral,1
7,12/13/2023,2_1,NaN,L4,Rostral,1
8,12/13/2023,3_1,NaN,L4,Caudal,2
9,12/13/2023,3_2,NaN,L2/3,Caudal,2


In [5]:
def get_shuffle_filename(slic, loc):
    return str(slic) + "_" + str(loc) + "shuffle.txt"

def get_barrel_roi_filename(slic):
    return str(slic) + "_barrel_rois.dat"

# locate all files
data_map = {}
for subdir, dirs, files in os.walk(data_dir):
    if 'hidden' in subdir:
        continue
    if not ('dic' in dirs and 'selected_zda' in dirs):
        continue
    print("\n", subdir)
    dic_dir = subdir + "/dic/"
    selected_zda_dir = subdir + "/selected_zda/"
    data_map[subdir] = {}
    for zda_file in os.listdir(selected_zda_dir):
        if zda_file.endswith('.zda'):
            slr_id = zda_file.split('.')[0]
            slic_id, loc_id, rec_id = [int(x) for x in slr_id.split("_")]
            print(slic_id, loc_id, rec_id)
            if slic_id not in data_map[subdir]:
                # find ROI file and index it
                data_map[subdir][slic_id] = {'roi_file': subdir + "/" + get_barrel_roi_filename(slic_id)}
            if loc_id not in data_map[subdir][slic_id]:
                # find shuffle file and index it
                data_map[subdir][slic_id][loc_id] = {'zda_files': [],
                                                     'shuffle_file': subdir + "/" +  get_shuffle_filename(slic_id, loc_id)}
            data_map[subdir][slic_id][loc_id]['zda_files'].append(selected_zda_dir + "/" + zda_file)  # appends in ascending order


 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/scnn1a-ppr/12-08-23
3 2 2
3 2 3
3 2 4
3 2 5
3 2 6
3 2 7
3 2 8
3 2 9
3 2 10
3 2 11
3 2 12
3 2 13
3 2 14
3 2 15
3 2 16
3 2 17
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 1 9
4 1 10
4 1 11
4 1 12
4 1 13
4 1 14
4 1 15
4 1 16
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 1 8
5 1 9
5 1 10
5 1 11
5 1 12
5 1 13
5 1 14
5 1 15
5 1 16
5 1 17
5 2 2
5 2 3
5 2 4
5 2 5
5 2 6
5 2 7
5 2 8
5 2 9
5 2 10
5 2 11
5 2 12
5 2 13
5 2 14
5 2 15
5 2 16
5 2 17
6 1 4
6 1 5
6 1 6
6 1 7
6 1 8
6 1 9
6 1 10
6 1 11
6 1 12
6 1 13
6 1 14
6 1 15
6 1 16
6 1 17
6 1 18
6 1 19
6 2 1
6 2 2
6 2 3
6 2 4
6 2 5
6 2 6
6 2 7
6 2 8
6 2 9
6 2 10
6 2 11
6 2 12
6 2 13
6 2 14
6 2 15
6 2 16

 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/scnn1a-ppr/12-13-23
1 4 2
1 4 3
1 4 4
1 4 5
1 4 6
1 4 7
1 4 8
1 4 9
1 4 10
1 4 11
1 4 12
1 4 13
1 4 14
1 4 15
1 4 16
1 4 17
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 1 8
2 1 9
2 1 10
2 1 11
2 1 12
2 1 13
2 1 14
2 1 15
2 1 16
3 1 2
3 1 3
3 1

In [6]:
# go through the IPIs in their shuffled order (ascending recordings) 
# and extract Amplitude, half_width, latency for pulse 1 and pulse 2 for each ROI
for subdir in data_map:
    aPhz = AutoPhotoZ(data_dir=subdir)
    print(subdir)
    for slic_id in data_map[subdir]:

        # open roi_file for whole slice
        if enable_photoZ_interact:
            roi_file = data_map[subdir][slic_id]['roi_file']
            aPhz.select_roi_tab()
            aPhz.open_roi_file(roi_file)

        for loc_id in data_map[subdir][slic_id]:
            if type(loc_id) == str:
                continue
            slic_loc_id = str(slic_id) + "_" + str(loc_id)

            ipi_file = data_map[subdir][slic_id][loc_id]['shuffle_file']
            with open(ipi_file, 'r') as f:
                lines = f.readlines() 
            ipi_order = [int(x) for x in lines]
            # open zda files in order
            i_ipi = -1
            for zda_file in data_map[subdir][slic_id][loc_id]['zda_files']:
                if enable_photoZ_interact:
                    aPhz.open_zda_file(zda_file)
                i_ipi += 1
                try:
                    ipi = ipi_order[i_ipi]
                except Exception as e:
                    print(e)
                    print("Not enough IPIs for how many recordings we have! Last IPI:", ipi, "Check", subdir)
                print(slic_id, loc_id, "IPI:", ipi, "ms for recording", zda_file.split("/")[-1])
                
                for i_pulse in [1, 2]:
                    if enable_photoZ_interact:
                        measure_window_start = 96 + (i_pulse - 1) * ipi * 2
                        aPhz.set_measure_window(measure_window_start, None)

                        # change polynomial skip window too to match measure window
                        aPhz.set_polynomial_skip_window(measure_window_start)

                    # take Amplitude, half_width, latency and label which pulse
                    # save SNR values from PhotoZ
                    snr_filename = subdir + "/" + "SNR_" + slic_loc_id + "_ipi" + str(ipi) + "_p" + str(i_pulse) + ".dat"
                    if enable_photoZ_interact:
                        aPhz.select_SNR_trace_value()
                        aPhz.save_trace_values(snr_filename)

                    # save half-width values from PhotoZ
                    hw_filename = subdir + "/" + "hw_" + slic_loc_id + "_ipi" + str(ipi) + "_p" + str(i_pulse) + ".dat" 
                    if enable_photoZ_interact:
                        aPhz.select_half_width_trace_value()
                        aPhz.save_trace_values(hw_filename)

                    # save amp values from PhotoZ
                    amp_filename = subdir + "/" + "amp_" + slic_loc_id + "_ipi" + str(ipi) + "_p" + str(i_pulse) + ".dat"
                    if enable_photoZ_interact:
                        aPhz.select_maxamp_trace_value()
                        aPhz.save_trace_values(amp_filename)
                        
                    # save latency values from PhotoZ
                    lat_filename = subdir + "/" + "lat_" + slic_loc_id + "_ipi" + str(ipi) + "_p" + str(i_pulse) + ".dat"
                    if enable_photoZ_interact:
                        aPhz.select_latency_trace_value()
                        aPhz.save_trace_values(lat_filename)


                    if ipi not in data_map[subdir][slic_id][loc_id]:
                        data_map[subdir][slic_id][loc_id][ipi] = {}
                    data_map[subdir][slic_id][loc_id][ipi][i_pulse] = {'amp': amp_filename,
                                                                       'hw': hw_filename,
                                                                       'snr': snr_filename,
                                                                       'lat': lat_filename}
            if i_ipi < len(ipi_order) - 1:
                print("Not enough recordings for how many IPIs we have! Last IPI:", ipi, "Check", subdir)
                        

C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/scnn1a-ppr/12-08-23
3 2 IPI: 10 ms for recording 03_02_02.zda
3 2 IPI: 20 ms for recording 03_02_03.zda
3 2 IPI: 30 ms for recording 03_02_04.zda
3 2 IPI: 40 ms for recording 03_02_05.zda
3 2 IPI: 50 ms for recording 03_02_06.zda
3 2 IPI: 60 ms for recording 03_02_07.zda
3 2 IPI: 70 ms for recording 03_02_08.zda
3 2 IPI: 80 ms for recording 03_02_09.zda
3 2 IPI: 90 ms for recording 03_02_10.zda
3 2 IPI: 100 ms for recording 03_02_11.zda
3 2 IPI: 110 ms for recording 03_02_12.zda
3 2 IPI: 120 ms for recording 03_02_13.zda
3 2 IPI: 130 ms for recording 03_02_14.zda
3 2 IPI: 140 ms for recording 03_02_15.zda
3 2 IPI: 150 ms for recording 03_02_16.zda
3 2 IPI: 5 ms for recording 03_02_17.zda
4 1 IPI: 5 ms for recording 04_01_01.zda
4 1 IPI: 10 ms for recording 04_01_02.zda
4 1 IPI: 20 ms for recording 04_01_03.zda
4 1 IPI: 30 ms for recording 04_01_04.zda
4 1 IPI: 40 ms for recording 04_01_05.zda
4 1 IPI: 50 ms for recording 04_01_06.z

1 3 IPI: 60 ms for recording 01_03_12.zda
1 3 IPI: 100 ms for recording 01_03_13.zda
1 3 IPI: 20 ms for recording 01_03_14.zda
1 3 IPI: 10 ms for recording 01_03_15.zda
1 3 IPI: 30 ms for recording 01_03_16.zda
2 1 IPI: 5 ms for recording 02_01_01.zda
2 1 IPI: 80 ms for recording 02_01_02.zda
2 1 IPI: 110 ms for recording 02_01_03.zda
2 1 IPI: 20 ms for recording 02_01_04.zda
2 1 IPI: 60 ms for recording 02_01_05.zda
2 1 IPI: 90 ms for recording 02_01_06.zda
2 1 IPI: 30 ms for recording 02_01_07.zda
2 1 IPI: 70 ms for recording 02_01_08.zda
2 1 IPI: 130 ms for recording 02_01_09.zda
2 1 IPI: 140 ms for recording 02_01_10.zda
2 1 IPI: 40 ms for recording 02_01_11.zda
2 1 IPI: 10 ms for recording 02_01_12.zda
2 1 IPI: 120 ms for recording 02_01_13.zda
2 1 IPI: 100 ms for recording 02_01_14.zda
2 1 IPI: 150 ms for recording 02_01_15.zda
2 1 IPI: 50 ms for recording 02_01_16.zda
3 1 IPI: 5 ms for recording 03_01_05.zda
3 1 IPI: 120 ms for recording 03_01_06.zda
3 1 IPI: 130 ms for recordin

In [7]:
# load all these amp, hw, snr, lat into a dataframe
def read_dat_trace_file(filename):
    data_arr = pd.read_csv(filename,
                                 sep='\t',
                                 header=None,
                                 names=['ROI',  'Value'])
    return data_arr

snr_dfs = []
for subdir in data_map:
    
    # extract date from subdir so we can look up metadata from master_df
    date = subdir.split("/")[-1].split('-')
    date = [int(x) for x in date]
    date[-1] += 2000
    date = "/".join([str(x) for x in date])
    metadata_date = master_df[master_df['Date'] == date]
    
    for slic_id in data_map[subdir]:
        for loc_id in data_map[subdir][slic_id]:
            if type(loc_id) == str:
                continue
            slic_loc_id = str(slic_id) + "_" + str(loc_id)
            metadata = metadata_date[metadata_date['Slice_Loc'] == slic_loc_id]
            medial_dir = metadata['Medial Dir'].item()
            stim_layer = metadata['Stim Layer'].item()
            caudal_rostral = metadata['Caudal/Rostral'].item()
            home_barrel = int(metadata['Home Barrel'].item())

            for ipi in data_map[subdir][slic_id][loc_id]:
                if type(ipi) == str:
                    continue
                for i_pulse in data_map[subdir][slic_id][loc_id][ipi]:
                    dm = data_map[subdir][slic_id][loc_id][ipi][i_pulse]
                    
                    try:
                        amp = read_dat_trace_file(dm['amp'])
                        hw = read_dat_trace_file(dm['hw'])
                        snr = read_dat_trace_file(dm['snr'])
                        lat = read_dat_trace_file(dm['lat'])
                    except Exception as e:
                        print(e)
                        continue
                        
                    snr = snr.join(amp, rsuffix='_amp')
                    snr = snr.join(hw, rsuffix='_hw')
                    snr = snr.join(lat, rsuffix='_lat')
                    snr.insert(4, 'Pulse', [i_pulse for _ in range(len(snr['ROI']))])
                    snr.insert(5, 'IPI', [ipi for _ in range(len(snr['ROI']))])
                    snr.insert(6, 'Medial Dir', [medial_dir for _ in range(len(snr['ROI']))])
                    snr.insert(7, 'Stim Layer', [stim_layer for _ in range(len(snr['ROI']))])
                    snr.insert(8, 'Caudal/Rostral', [caudal_rostral for _ in range(len(snr['ROI']))])
                    snr.insert(9, 'Home Barrel', [home_barrel for _ in range(len(snr['ROI']))])
                    snr.insert(10, 'Date', [date for _ in range(len(snr['ROI']))])
                    snr.insert(11, 'Slice_Loc', [slic_loc_id for _ in range(len(snr['ROI']))])
                    snr = snr[pd.to_numeric(snr.Value, errors='coerce').notnull()]
                    snr_dfs.append(snr)
        
pre_filter_snr = pd.concat(snr_dfs)
pre_filter_snr = pre_filter_snr[['Date', 'Slice_Loc', 'ROI', 'Value', 'Value_amp', 'Value_hw', 
                                 'Value_lat', 'Pulse', 'IPI',
                                 'Medial Dir', 'Stim Layer', 'Caudal/Rostral', 'Home Barrel']]
pre_filter_snr = pre_filter_snr.rename(columns={'Value': "SNR",
                                                'Value_amp': "Amp", 
                                                'Value_hw' : "Halfwidth", 
                                                'Value_lat': "Latency"})
pre_filter_snr["Amp"] = pd.to_numeric(pre_filter_snr["Amp"])
pre_filter_snr['PPR'] = ''
pre_filter_snr

,Date,Slice_Loc,ROI,SNR,Amp,Halfwidth,Latency,Pulse,IPI,Medial Dir,Stim Layer,Caudal/Rostral,Home Barrel,PPR
0,12/8/2023,3_2,1,1.792070,0.928909,-104.79500,52.4487,1,10,up,L2/3,Caudal,2,
1,12/8/2023,3_2,2,4.903530,2.221310,5.67150,51.5911,1,10,up,L2/3,Caudal,2,
2,12/8/2023,3_2,3,1.835520,1.020520,-104.42200,52.2619,1,10,up,L2/3,Caudal,2,
0,12/8/2023,3_2,1,1.775360,0.926149,7.76580,60.0771,2,10,up,L2/3,Caudal,2,
1,12/8/2023,3_2,2,4.051870,1.837380,4.46569,61.5711,2,10,up,L2/3,Caudal,2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,12/19/2023,4_1,2,5.464650,0.483628,6.71430,80.1991,2,30,NaN,L4,Rostral,2,
0,12/19/2023,4_1,1,0.910626,0.043237,1.53890,53.7013,1,20,NaN,L4,Rostral,2,
1,12/19/2023,4_1,2,3.893600,0.373637,4.81526,49.8283,1,20,NaN,L4,Rostral,2,
0,12/19/2023,4_1,1,2.101880,0.096144,4.08306,70.2512,2,20,NaN,L4,Rostral,2,


In [12]:
# Now compute PPR = Amp(pulse 2) / Amp(pulse 1). They must match Date, Slice_Loc, and ROI 
for date in pre_filter_snr['Date'].unique():
    pfs_date = pre_filter_snr[pre_filter_snr["Date"] == date]
    for slic_loc in pfs_date['Slice_Loc'].unique():
        pfs_sl = pfs_date[pfs_date["Slice_Loc"] == slic_loc]
        for roi in pfs_sl['ROI'].unique():
            pfs_roi = pfs_sl[pfs_sl['ROI'] == roi]
            for ipi in pfs_roi['IPI'].unique():
                pfs_ipi = pfs_roi[pfs_roi['IPI'] == ipi]
                assert len(pfs_ipi) == 2  # should always be exactly 2 (otherwise can't compute PPR)

                amp1 = pfs_ipi[pfs_ipi['Pulse'] == 1]['Amp'].item()
                ppr = 0
                if amp1 != 0:
                    amp2 = pfs_ipi[pfs_ipi['Pulse'] == 2]['Amp'].item()
                    ppr = amp2 / amp1
                pre_filter_snr.loc[(pre_filter_snr['Date'] == date) &
                               (pre_filter_snr['Slice_Loc'] == slic_loc) &
                               (pre_filter_snr['ROI'] == roi), 
                                   'PPR'] = ppr
                    
pre_filter_snr

,Date,Slice_Loc,ROI,SNR,Amp,Halfwidth,Latency,Pulse,IPI,Medial Dir,Stim Layer,Caudal/Rostral,Home Barrel,PPR
0,12/8/2023,3_2,1,1.792070,0.928909,-104.79500,52.4487,1,10,up,L2/3,Caudal,2,1.009584
1,12/8/2023,3_2,2,4.903530,2.221310,5.67150,51.5911,1,10,up,L2/3,Caudal,2,1.004662
2,12/8/2023,3_2,3,1.835520,1.020520,-104.42200,52.2619,1,10,up,L2/3,Caudal,2,0.990825
0,12/8/2023,3_2,1,1.775360,0.926149,7.76580,60.0771,2,10,up,L2/3,Caudal,2,1.009584
1,12/8/2023,3_2,2,4.051870,1.837380,4.46569,61.5711,2,10,up,L2/3,Caudal,2,1.004662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,12/19/2023,4_1,2,5.464650,0.483628,6.71430,80.1991,2,30,NaN,L4,Rostral,2,1.313408
0,12/19/2023,4_1,1,0.910626,0.043237,1.53890,53.7013,1,20,NaN,L4,Rostral,2,2.223664
1,12/19/2023,4_1,2,3.893600,0.373637,4.81526,49.8283,1,20,NaN,L4,Rostral,2,1.313408
0,12/19/2023,4_1,1,2.101880,0.096144,4.08306,70.2512,2,20,NaN,L4,Rostral,2,2.223664


In [13]:
# now some filtering
all_snr = pre_filter_snr[pre_filter_snr['SNR'] > snr_cutoff_signal]
all_snr = all_snr[all_snr['Halfwidth'] > 0]
all_snr = all_snr[all_snr['Halfwidth'] < 15]
all_snr = all_snr[all_snr['Amp'] > amp_cutoff]